- Sentence Extraction

In [ ]:
import pandas as pd
import Example
import sqlite3
# 신조어 목록 불러오기
df = pd.read_excel('new_words_var.xlsx')
new_words = df[df.y == 1].iloc[:,1].drop_duplicates(keep='first')

# 전체 데이터 합치기
conn = sqlite3.connect('Total_Ent.db')
df1 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
conn = sqlite3.connect('HP.db')
df2 = pd.read_sql('SELECT head FROM head', conn)
conn.close()
head = df1.append(df2)

# 불용어처리 없는 sentence 추출
sent = Example.extract_sent(head, new_words)
sent = pd.DataFrame.from_dict(sent, orient='index')
conn = sqlite3.connect('sent.db')
sent.to_sql('sent', conn)

- Generating Model using ltokenizer

In [1]:
import sqlite3
import pandas as pd
from collections import defaultdict
import Example
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('News_sent.db') # sent.db 연결
temp = pd.read_sql('SELECT * FROM sent', conn)  # sent.db에서 전체 데이터를 temp에 저장

full_sentence = list()
example_sentence = list()
for _ in range(len(temp['0'])):
    try:
        sent = temp['0'][_] # _번째 문장들
        sent = sent.split('  ') # _번째 문장들은 더블스페이스 단위로 분리
        sent = [_+'/' for _ in sent] # 분리된 문장을 '/' 로 구분
        words = {temp['index'][_]: 1.0} # tokenizer가 신어를 하나의 단어로 인식할 수 있도록, 신어를 1로 저장하는 dict 생성

        lgen = Example.generator_ltokenizer(words, sent) # ltokenizer사용 
        sentences = [[word for word in lgen.tokenizer(sentence)] for sentence in sent] # s
        lgen.train_word_model(sentences)
        X,y = lgen.create_var(sentences)
        lgen.model()
        lgen.model.fit(X, y, epochs=70, verbose=2)

        temp_sent = lgen.sentence_generation(temp['index'][_], 22) # 각단어에 해당하는 22 단어로 이루어진 문장 생성
        result = ''
        for _ in temp_sent: # temp_sent에서 /로 끝나면 break
            if _ == '/':
                break
            else:
                result = result + '' + _
        full_sentence.append(temp_sent)
        example_sentence.append(result)
    except:
        full_sentence.append('Error : {}'.format(temp['index'][_]))
        example_sentence.append('Error : {}'.format(temp['index'][_]))

training was done. used memory 0.331 Gby 0.329 Gb
all cohesion probabilities was computed. # words = 366
Epoch 1/70
54/54 - 10s - loss: 6.6421 - accuracy: 0.0655
Epoch 2/70
54/54 - 1s - loss: 6.0253 - accuracy: 0.0742
Epoch 3/70
54/54 - 1s - loss: 5.7853 - accuracy: 0.0742
Epoch 4/70
54/54 - 1s - loss: 5.6144 - accuracy: 0.0760
Epoch 5/70
54/54 - 1s - loss: 5.4014 - accuracy: 0.0899
Epoch 6/70
54/54 - 1s - loss: 5.1329 - accuracy: 0.1073
Epoch 7/70
54/54 - 1s - loss: 4.7951 - accuracy: 0.1410
Epoch 8/70
54/54 - 1s - loss: 4.4602 - accuracy: 0.1937
Epoch 9/70
54/54 - 1s - loss: 4.1528 - accuracy: 0.2471
Epoch 10/70
54/54 - 1s - loss: 3.8430 - accuracy: 0.2964
Epoch 11/70
54/54 - 1s - loss: 3.5515 - accuracy: 0.3527
Epoch 12/70
54/54 - 1s - loss: 3.2915 - accuracy: 0.4055
Epoch 13/70
54/54 - 1s - loss: 3.0705 - accuracy: 0.4292
Epoch 14/70
54/54 - 1s - loss: 2.8506 - accuracy: 0.4629
Epoch 15/70
54/54 - 1s - loss: 2.6548 - accuracy: 0.4959
Epoch 16/70
54/54 - 1s - loss: 2.4668 - accuracy

Epoch 2/70
303/303 - 10s - loss: 7.0235 - accuracy: 0.0801
Epoch 3/70
303/303 - 9s - loss: 6.6266 - accuracy: 0.1091
Epoch 4/70
303/303 - 9s - loss: 6.2508 - accuracy: 0.1286
Epoch 5/70
303/303 - 9s - loss: 5.9236 - accuracy: 0.1448
Epoch 6/70
303/303 - 8s - loss: 5.5748 - accuracy: 0.1612
Epoch 7/70
303/303 - 9s - loss: 5.2425 - accuracy: 0.1817
Epoch 8/70
303/303 - 9s - loss: 4.9322 - accuracy: 0.2056
Epoch 9/70
303/303 - 9s - loss: 4.5871 - accuracy: 0.2409
Epoch 10/70
303/303 - 9s - loss: 4.2392 - accuracy: 0.2780
Epoch 11/70
303/303 - 9s - loss: 3.9052 - accuracy: 0.3252
Epoch 12/70
303/303 - 9s - loss: 3.5632 - accuracy: 0.3850
Epoch 13/70
303/303 - 9s - loss: 3.2119 - accuracy: 0.4415
Epoch 14/70
303/303 - 10s - loss: 2.8815 - accuracy: 0.4918
Epoch 15/70
303/303 - 10s - loss: 2.5629 - accuracy: 0.5459
Epoch 16/70
303/303 - 9s - loss: 2.2770 - accuracy: 0.5943
Epoch 17/70
303/303 - 9s - loss: 2.0104 - accuracy: 0.6428
Epoch 18/70
303/303 - 9s - loss: 1.7644 - accuracy: 0.6783
Ep

Epoch 1/70
27/27 - 7s - loss: 5.9581 - accuracy: 0.0406
Epoch 2/70
27/27 - 0s - loss: 5.6398 - accuracy: 0.0489
Epoch 3/70
27/27 - 0s - loss: 5.3631 - accuracy: 0.0561
Epoch 4/70
27/27 - 0s - loss: 5.2002 - accuracy: 0.0728
Epoch 5/70
27/27 - 0s - loss: 4.9955 - accuracy: 0.1026
Epoch 6/70
27/27 - 0s - loss: 4.7736 - accuracy: 0.1432
Epoch 7/70
27/27 - 0s - loss: 4.5287 - accuracy: 0.1611
Epoch 8/70
27/27 - 0s - loss: 4.2999 - accuracy: 0.1909
Epoch 9/70
27/27 - 0s - loss: 4.0527 - accuracy: 0.2220
Epoch 10/70
27/27 - 0s - loss: 3.7869 - accuracy: 0.2697
Epoch 11/70
27/27 - 0s - loss: 3.5441 - accuracy: 0.3222
Epoch 12/70
27/27 - 0s - loss: 3.2885 - accuracy: 0.3640
Epoch 13/70
27/27 - 0s - loss: 3.0666 - accuracy: 0.4033
Epoch 14/70
27/27 - 0s - loss: 2.8417 - accuracy: 0.4463
Epoch 15/70
27/27 - 0s - loss: 2.6540 - accuracy: 0.4821
Epoch 16/70
27/27 - 0s - loss: 2.4636 - accuracy: 0.5155
Epoch 17/70
27/27 - 0s - loss: 2.3088 - accuracy: 0.5382
Epoch 18/70
27/27 - 0s - loss: 2.1716 - 

Epoch 1/70
3/3 - 6s - loss: 3.9278 - accuracy: 0.0435
Epoch 2/70
3/3 - 0s - loss: 3.8986 - accuracy: 0.1594
Epoch 3/70
3/3 - 0s - loss: 3.8779 - accuracy: 0.1739
Epoch 4/70
3/3 - 0s - loss: 3.8539 - accuracy: 0.1739
Epoch 5/70
3/3 - 0s - loss: 3.8263 - accuracy: 0.1739
Epoch 6/70
3/3 - 0s - loss: 3.7890 - accuracy: 0.1739
Epoch 7/70
3/3 - 0s - loss: 3.7399 - accuracy: 0.1739
Epoch 8/70
3/3 - 0s - loss: 3.6945 - accuracy: 0.1739
Epoch 9/70
3/3 - 0s - loss: 3.6414 - accuracy: 0.1739
Epoch 10/70
3/3 - 0s - loss: 3.5793 - accuracy: 0.1739
Epoch 11/70
3/3 - 0s - loss: 3.5047 - accuracy: 0.1739
Epoch 12/70
3/3 - 0s - loss: 3.4801 - accuracy: 0.1739
Epoch 13/70
3/3 - 0s - loss: 3.4631 - accuracy: 0.1739
Epoch 14/70
3/3 - 0s - loss: 3.3919 - accuracy: 0.1739
Epoch 15/70
3/3 - 0s - loss: 3.3729 - accuracy: 0.1739
Epoch 16/70
3/3 - 0s - loss: 3.3499 - accuracy: 0.1739
Epoch 17/70
3/3 - 0s - loss: 3.2836 - accuracy: 0.1739
Epoch 18/70
3/3 - 0s - loss: 3.3047 - accuracy: 0.1739
Epoch 19/70
3/3 - 0

Epoch 7/70
16/16 - 0s - loss: 4.8354 - accuracy: 0.1026
Epoch 8/70
16/16 - 0s - loss: 4.7074 - accuracy: 0.1026
Epoch 9/70
16/16 - 0s - loss: 4.5858 - accuracy: 0.1460
Epoch 10/70
16/16 - 0s - loss: 4.4286 - accuracy: 0.1617
Epoch 11/70
16/16 - 0s - loss: 4.2704 - accuracy: 0.1953
Epoch 12/70
16/16 - 0s - loss: 4.1312 - accuracy: 0.2189
Epoch 13/70
16/16 - 0s - loss: 3.9481 - accuracy: 0.2268
Epoch 14/70
16/16 - 0s - loss: 3.8263 - accuracy: 0.2446
Epoch 15/70
16/16 - 0s - loss: 3.6685 - accuracy: 0.2623
Epoch 16/70
16/16 - 0s - loss: 3.5416 - accuracy: 0.2722
Epoch 17/70
16/16 - 0s - loss: 3.4172 - accuracy: 0.2899
Epoch 18/70
16/16 - 0s - loss: 3.2906 - accuracy: 0.3353
Epoch 19/70
16/16 - 0s - loss: 3.1418 - accuracy: 0.3491
Epoch 20/70
16/16 - 0s - loss: 3.0104 - accuracy: 0.3925
Epoch 21/70
16/16 - 0s - loss: 2.8804 - accuracy: 0.4083
Epoch 22/70
16/16 - 0s - loss: 2.7521 - accuracy: 0.4339
Epoch 23/70
16/16 - 0s - loss: 2.6183 - accuracy: 0.4596
Epoch 24/70
16/16 - 0s - loss: 2.4

...

Epoch 43/70
4/4 - 0s - loss: 1.8022 - accuracy: 0.5913
Epoch 44/70
4/4 - 0s - loss: 1.6978 - accuracy: 0.6087
Epoch 45/70
4/4 - 0s - loss: 1.6110 - accuracy: 0.6348
Epoch 46/70
4/4 - 0s - loss: 1.5784 - accuracy: 0.6435
Epoch 47/70
4/4 - 0s - loss: 1.5210 - accuracy: 0.6783
Epoch 48/70
4/4 - 0s - loss: 1.5143 - accuracy: 0.6783
Epoch 49/70
4/4 - 0s - loss: 1.4160 - accuracy: 0.7217
Epoch 50/70
4/4 - 0s - loss: 1.3196 - accuracy: 0.7652
Epoch 51/70
4/4 - 0s - loss: 1.3277 - accuracy: 0.7217
Epoch 52/70
4/4 - 0s - loss: 1.2425 - accuracy: 0.7391
Epoch 53/70
4/4 - 0s - loss: 1.2593 - accuracy: 0.7217
Epoch 54/70
4/4 - 0s - loss: 1.1607 - accuracy: 0.8261
Epoch 55/70
4/4 - 0s - loss: 1.1068 - accuracy: 0.8261
Epoch 56/70
4/4 - 0s - loss: 1.0776 - accuracy: 0.7913
Epoch 57/70
4/4 - 0s - loss: 1.0098 - accuracy: 0.8435
Epoch 58/70
4/4 - 0s - loss: 0.9844 - accuracy: 0.8261
Epoch 59/70
4/4 - 0s - loss: 0.9900 - accuracy: 0.8609
Epoch 60/70
4/4 - 0s - loss: 0.8946 - accuracy: 0.8609
Epoch 61/7

Epoch 49/70
3/3 - 0s - loss: 2.0096 - accuracy: 0.5269
Epoch 50/70
3/3 - 0s - loss: 1.9193 - accuracy: 0.6022
Epoch 51/70
3/3 - 0s - loss: 1.9188 - accuracy: 0.5699
Epoch 52/70
3/3 - 0s - loss: 1.7355 - accuracy: 0.7097
Epoch 53/70
3/3 - 0s - loss: 1.6732 - accuracy: 0.6774
Epoch 54/70
3/3 - 0s - loss: 1.6553 - accuracy: 0.7204
Epoch 55/70
3/3 - 0s - loss: 1.6627 - accuracy: 0.6989
Epoch 56/70
3/3 - 0s - loss: 1.6380 - accuracy: 0.6129
Epoch 57/70
3/3 - 0s - loss: 1.5106 - accuracy: 0.6882
Epoch 58/70
3/3 - 0s - loss: 1.4610 - accuracy: 0.6989
Epoch 59/70
3/3 - 0s - loss: 1.4103 - accuracy: 0.7312
Epoch 60/70
3/3 - 0s - loss: 1.3102 - accuracy: 0.8172
Epoch 61/70
3/3 - 0s - loss: 1.2726 - accuracy: 0.7957
Epoch 62/70
3/3 - 0s - loss: 1.2289 - accuracy: 0.7634
Epoch 63/70
3/3 - 0s - loss: 1.1632 - accuracy: 0.8280
Epoch 64/70
3/3 - 0s - loss: 1.2137 - accuracy: 0.7849
Epoch 65/70
3/3 - 0s - loss: 1.1222 - accuracy: 0.8817
Epoch 66/70
3/3 - 0s - loss: 1.1554 - accuracy: 0.7849
Epoch 67/7

Epoch 51/70
149/149 - 3s - loss: 0.1999 - accuracy: 0.9553
Epoch 52/70
149/149 - 3s - loss: 0.1788 - accuracy: 0.9591
Epoch 53/70
149/149 - 3s - loss: 0.1735 - accuracy: 0.9591
Epoch 54/70
149/149 - 3s - loss: 0.1730 - accuracy: 0.9608
Epoch 55/70
149/149 - 3s - loss: 0.1698 - accuracy: 0.9599
Epoch 56/70
149/149 - 3s - loss: 0.1715 - accuracy: 0.9584
Epoch 57/70
149/149 - 3s - loss: 0.1534 - accuracy: 0.9631
Epoch 58/70
149/149 - 3s - loss: 0.1504 - accuracy: 0.9618
Epoch 59/70
149/149 - 3s - loss: 0.1508 - accuracy: 0.9627
Epoch 60/70
149/149 - 3s - loss: 0.1404 - accuracy: 0.9627
Epoch 61/70
149/149 - 3s - loss: 0.1463 - accuracy: 0.9599
Epoch 62/70
149/149 - 3s - loss: 0.1553 - accuracy: 0.9582
Epoch 63/70
149/149 - 3s - loss: 0.1417 - accuracy: 0.9605
Epoch 64/70
149/149 - 3s - loss: 0.1453 - accuracy: 0.9614
Epoch 65/70
149/149 - 3s - loss: 0.1295 - accuracy: 0.9635
Epoch 66/70
149/149 - 3s - loss: 0.1309 - accuracy: 0.9631
Epoch 67/70
149/149 - 3s - loss: 0.1260 - accuracy: 0.96

Epoch 51/70
21/21 - 0s - loss: 0.5963 - accuracy: 0.9189
Epoch 52/70
21/21 - 0s - loss: 0.5565 - accuracy: 0.9294
Epoch 53/70
21/21 - 0s - loss: 0.5455 - accuracy: 0.9204
Epoch 54/70
21/21 - 0s - loss: 0.5261 - accuracy: 0.9264
Epoch 55/70
21/21 - 0s - loss: 0.5207 - accuracy: 0.9264
Epoch 56/70
21/21 - 0s - loss: 0.4957 - accuracy: 0.9354
Epoch 57/70
21/21 - 0s - loss: 0.4798 - accuracy: 0.9264
Epoch 58/70
21/21 - 0s - loss: 0.4674 - accuracy: 0.9219
Epoch 59/70
21/21 - 0s - loss: 0.4385 - accuracy: 0.9324
Epoch 60/70
21/21 - 0s - loss: 0.4434 - accuracy: 0.9324
Epoch 61/70
21/21 - 0s - loss: 0.4248 - accuracy: 0.9429
Epoch 62/70
21/21 - 0s - loss: 0.4023 - accuracy: 0.9429
Epoch 63/70
21/21 - 0s - loss: 0.3948 - accuracy: 0.9369
Epoch 64/70
21/21 - 0s - loss: 0.3865 - accuracy: 0.9369
Epoch 65/70
21/21 - 0s - loss: 0.3714 - accuracy: 0.9474
Epoch 66/70
21/21 - 0s - loss: 0.3620 - accuracy: 0.9369
Epoch 67/70
21/21 - 0s - loss: 0.3479 - accuracy: 0.9505
Epoch 68/70
21/21 - 0s - loss: 

3/3 - 0s - loss: 1.5047 - accuracy: 0.7791
Epoch 54/70
3/3 - 0s - loss: 1.4833 - accuracy: 0.7674
Epoch 55/70
3/3 - 0s - loss: 1.3958 - accuracy: 0.7791
Epoch 56/70
3/3 - 0s - loss: 1.3821 - accuracy: 0.8488
Epoch 57/70
3/3 - 0s - loss: 1.3454 - accuracy: 0.7558
Epoch 58/70
3/3 - 0s - loss: 1.3032 - accuracy: 0.8256
Epoch 59/70
3/3 - 0s - loss: 1.2411 - accuracy: 0.7674
Epoch 60/70
3/3 - 0s - loss: 1.2475 - accuracy: 0.8023
Epoch 61/70
3/3 - 0s - loss: 1.1853 - accuracy: 0.8488
Epoch 62/70
3/3 - 0s - loss: 1.1982 - accuracy: 0.8605
Epoch 63/70
3/3 - 0s - loss: 1.0203 - accuracy: 0.9186
Epoch 64/70
3/3 - 0s - loss: 0.9968 - accuracy: 0.9419
Epoch 65/70
3/3 - 0s - loss: 1.0080 - accuracy: 0.8372
Epoch 66/70
3/3 - 0s - loss: 0.9987 - accuracy: 0.8953
Epoch 67/70
3/3 - 0s - loss: 0.9922 - accuracy: 0.8488
Epoch 68/70
3/3 - 0s - loss: 0.8811 - accuracy: 0.8953
Epoch 69/70
3/3 - 0s - loss: 0.8965 - accuracy: 0.8837
Epoch 70/70
3/3 - 0s - loss: 0.8446 - accuracy: 0.9186


- White Space Model Using N-Gram

In [4]:
def ngram_umjeol(s, N=2, opt=False):
    ngram = list()
    for i in range(len(s)-(N-1)):
        ngram.append(''.join(s[i:i+N]) if opt else tuple(s[i:i+N]))
    return ngram

def find_key(key, base):
    return list(filter(lambda keys: keys[:len(key)]==key, base.keys()))

def find_prob(key, freq, base):
    return [(_, base[_], base[_]/freq if freq > 0 else 0) for _ in find_key(key,base)]

def find_top5(key, freq, base):
    return sorted(find_prob(key,freq, base), key=lambda _ : _[-1], reverse=True)[:5]

def ngram(key, base1, base2):
    nextkey = find_top5(key, base1[key], base2)
    return [_[0] for _ in nextkey]

In [13]:
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
sent = pd.read_excel('example4.xlsx') # 생성한 예제 문장 불러오기

conn = sqlite3.connect('News_sent.db') # 예제 문장이 만들어진 DataBase 연결
corpus = pd.read_sql('SELECT * FROM sent', conn) # 전체 문장 Dataset 가져오기

ws_sent = list() # 띄어쓰기가 완료된 문장 넣어둘 list 생성
for _ in tqdm_notebook(sent[0]):
    try:
        temp = _.split(' ') # 띄어쓰기 단위로 잘라서 temp에 저장
        sentences = corpus[corpus['index'] == temp[0]]['0'].iloc[0] # 신어에 해당하는 전체 문장 가져오기

        # 예제 문장 길이에 해당하는 GRAM을 list로 만들기
        GRAM_list = [defaultdict(lambda:0) for i in range(len(_))]
        for i in range(len(_)):
            for j in ngram_umjeol(sentences, i+1, True):
                GRAM_list[i][j] += 1

        etemp = temp[0] # 띄어쓰기가 진행될 식별자 etemp 생성
        for i in range(len(temp)-1):
            base1 = len(temp[i])
            base2 = len(temp[i]+temp[i+1])
            # 둘을 띄어쓰기 하지 않고 붙인게 자주쓰는 표현 5개 안에 있으면 그대로 붙이기
            if temp[i]+temp[i+1] in ngram(temp[i], GRAM_list[base1-1], GRAM_list[base2-1]):
                etemp += temp[i+1]
            # 그렇지 않으면 띄어쓰기 하기
            else:
                etemp += ' ' + temp[i+1]
        ws_sent.append(etemp) # 띄어쓰기 완료된 문장 ws_sent에 저장
    except:
        ws_sent.append(_)
    
pd.DataFrame(ws_sent).to_excel('News_wsexample.xlsx') # excel에 저장

C:\Users\gozj3\anaconda3\lib\site-packages\ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()
